In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix,classification_report

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


In [3]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(128, 128),
                                                 batch_size=32,
                                                 class_mode='categorical')

Found 4103 images belonging to 3 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='categorical')

Found 1806 images belonging to 3 classes.


In [6]:
classifier = Sequential()

In [7]:
classifier.add(Convolution2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

In [8]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [9]:
classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [10]:
classifier.add(Flatten())

In [11]:
classifier.add(Dense(units = 128, activation = 'relu',kernel_initializer="uniform"))
classifier.add(Dropout(0.2))
#classifier.add(Dense(units = 32, activation = 'relu',kernel_initializer="uniform"))
#classifier.add(Dropout(0.2))
classifier.add(Dense(units = 3, activation = 'softmax'))

In [12]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
classifier.fit_generator(training_set,
                        samples_per_epoch=250,
                        nb_epoch=5,
                        validation_data=test_set,
                        nb_val_samples=100)

C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=7, epochs=5, validation_steps=100)`
  """


Epoch 1/5
7/7 [==============================] - 35s 5s/step - loss: 1.9749 - acc: 0.4196 - val_loss: 0.9740 - val_acc: 0.3693
Epoch 2/5
7/7 [==============================] - 33s 5s/step - loss: 0.8846 - acc: 0.5938 - val_loss: 0.6443 - val_acc: 0.7298
Epoch 3/5
7/7 [==============================] - 33s 5s/step - loss: 0.6272 - acc: 0.6830 - val_loss: 0.5196 - val_acc: 0.7769
Epoch 4/5
7/7 [==============================] - 35s 5s/step - loss: 0.5559 - acc: 0.7545 - val_loss: 0.5254 - val_acc: 0.7132
Epoch 5/5
7/7 [==============================] - 33s 5s/step - loss: 0.4139 - acc: 0.8214 - val_loss: 0.3439 - val_acc: 0.8726


In [14]:
classifier.save('model.h5')

In [590]:
from keras.models import load_model
model = load_model('model.h5')

In [726]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('singlePrediction/ClipOrPhotoOrSketch16.jpg', target_size=(128, 128))

In [727]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [728]:
result[0][0]

1.0

In [729]:
result[0]

array([1.0000000e+00, 1.3659629e-14, 0.0000000e+00], dtype=float32)

In [730]:
training_set.class_indices

{'Photos': 0, 'clipart': 1, 'sketch': 2}

In [697]:
pc=result[0].tolist().index(np.amax(result[0]))
pc

0

In [698]:
training_set.class_indices
if pc == 0: 
    prediction = 'Photos'
elif pc == 1: 
    prediction = 'clipart'
else:
    prediction = 'sketch'

In [699]:
prediction

'Photos'